In [1]:
#Prior to execute this notebook, please make sure you have already cloned the sample data
#!git clone https://github.com/xieyongliang/all-in-one-ai-sample-data.git -t ../../..

In [31]:
import os
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel

In [2]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [14]:
!rm -rf sourcedir.tar.gz
!tar czvf sourcedir.tar.gz *

evaluate.py
finetune.py
inference.py
model.py
paddlenlp.ipynb
requirements.txt
uie_predictor.py
utils.py


In [15]:
source_dir = 's3://{0}/{1}/source/'.format(bucket, 'paddlenlp')
!aws s3 cp sourcedir.tar.gz $source_dir

upload: ./sourcedir.tar.gz to s3://sagemaker-ap-east-1-034068151705/paddlenlp/source/sourcedir.tar.gz


In [16]:
train_dir = 's3://{0}/{1}/data/train'.format(bucket, 'paddlenlp')
!aws s3 cp ../../../all-in-one-ai-sample-data/paddlenlp/train/ $train_dir --recursive

upload: ../../../all-in-one-ai-sample-data/paddlenlp/train/test.txt to s3://sagemaker-ap-east-1-034068151705/paddlenlp/data/train/test.txt
upload: ../../../all-in-one-ai-sample-data/paddlenlp/train/train.txt to s3://sagemaker-ap-east-1-034068151705/paddlenlp/data/train/train.txt
upload: ../../../all-in-one-ai-sample-data/paddlenlp/train/dev.txt to s3://sagemaker-ap-east-1-034068151705/paddlenlp/data/train/dev.txt


In [17]:
job_name = None
entry_point = 'finetune.py'
source_dir = '{0}sourcedir.tar.gz'.format(source_dir)
git_config = None
role = role
hyperparameters = {
    'train_path': '/opt/ml/input/data/dataset/train.txt',
    'dev_path': '/opt/ml/input/data/dataset/dev.txt', 
    'save_dir': '/opt/ml/model',                 
    'batch_size' : 16, 
    'learning_rate' : 1e-5, 
    'max_seq_len' : 512,
    'num_epochs' : 100,
    'seed' : 1000,
    'logging_steps': 10,
    'valid_steps': 100,
    'device': 'gpu',
    'model': 'uie-base'
}
framework_version = '1.9.0'
py_version = 'py38'
instance_type = 'ml.g4dn.16xlarge'
instance_count = 1
inputs = {
    'dataset': train_dir
}

In [18]:
print(source_dir)

s3://sagemaker-ap-east-1-034068151705/paddlenlp/source/sourcedir.tar.gz


In [19]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = source_dir,
    git_config = git_config,
    role = role,
    debugger_hook_config = False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [ ]:
estimator.fit(inputs, job_name = job_name)

2022-09-13 07:56:26 Starting - Starting the training job...
2022-09-13 07:56:42 Starting - Preparing the instances for trainingProfilerReport-1663055786: InProgress
......
2022-09-13 07:57:51 Downloading - Downloading input data...
2022-09-13 07:58:12 Training - Downloading the training image.....................
2022-09-13 08:01:53 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-09-13 08:01:45,239 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-09-13 08:01:45,263 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-09-13 08:01:45,268 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-09-13 08:01:45,549 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r require

In [66]:
sample_dir ='../../../all-in-one-ai-sample-data/paddlenlp/inference/sample'

In [67]:
from os import listdir
from os.path import isfile, join

files = [f for f in listdir(sample_dir) if isfile(join(sample_dir, f))]

In [70]:
inputs  = open(join(sample_dir, files[0]), 'r').read()

In [72]:
training_job_name = estimator.latest_training_job.name

In [73]:
model_name = None
model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name)
entry_point = 'inference.py'
framework_version = '1.9.0'
py_version = 'py38'
model_environment = {'schema': '["出发地", "目的地", "费用", "时间"]'}

In [74]:
model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [45]:
endpoint_name = None
instance_type = 'ml.m5.xlarge'
instance_count = 1

In [46]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

----------!

In [54]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(
    {"inputs":"上海虹桥高铁到杭州时间是9月24日费用是73元"}
)

{'result': [{'出发地': [{'text': '上海',
     'start': 0,
     'end': 2,
     'probability': 0.979479968547821}],
   '目的地': [{'text': '杭州',
     'start': 7,
     'end': 9,
     'probability': 0.9992395639419556}],
   '费用': [{'text': '73',
     'start': 20,
     'end': 22,
     'probability': 0.560113251209259}],
   '时间': [{'text': '9月24日',
     'start': 12,
     'end': 17,
     'probability': 0.999651312828064}]}]}

In [ ]:
predictor.delete_endpoint()